In [1]:
import open3d as o3d
import numpy as np
import copy
import utils.alignment as al
import utils.filter as fltr

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Loading and aligning

In [2]:
source = o3d.io.read_point_cloud(r'point_clouds/BrokenMug.ply')
#source = fltr.filter_background(source)
target = o3d.io.read_point_cloud(r'point_clouds/BaseMug.ply')

In [3]:
# disturb source
for point in source.points:
    shift = np.random.uniform(low=-100, high=100, size=(3,))
    shift /= np.linalg.norm(shift) * (1/0.0001)
    point += shift

R = source.get_rotation_matrix_from_xyz((np.pi / 2, 0, 0))
source.rotate(R)
source.translate(np.random.uniform(low=-1, high=1, size=(3,)))

PointCloud with 25736 points.

In [4]:
transformation = al.global_registration(source, target)
source_temp = copy.deepcopy(source)
target_temp = copy.deepcopy(target)

transformation = transformation.copy()
print(transformation)
source_temp.transform(transformation)
source_temp.paint_uniform_color([0, 1, 0])
target_temp.paint_uniform_color([0, 0.651, 0.929])
o3d.visualization.draw_geometries([source_temp, target_temp])

[Open3D WARNING] Too few correspondences (1734) after mutual filter, fall back to original correspondences.
[[ 0.99999448  0.00228001  0.00241507 -0.20419237]
 [-0.00242384  0.00384904  0.99998965  0.65489674]
 [ 0.00227069 -0.99998999  0.00385454 -0.78496669]
 [ 0.          0.          0.          1.        ]]
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


### Comparing

In [5]:
threshold = 0.0075

mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(source_temp, depth=9)

vertices = o3d.core.Tensor(np.array(mesh.vertices), dtype=o3d.core.float32)
triangles = o3d.core.Tensor(np.array(mesh.triangles), dtype=o3d.core.uint32)

scene = o3d.t.geometry.RaycastingScene()
scene.add_triangles(vertices, triangles)

distances = []
defect_points = [] # (point, color)

for point in target_temp.points:
    query_point = o3d.core.Tensor([point], dtype=o3d.core.float32)
    distance = scene.compute_distance(query_point)
    if distance > threshold:
        if distance > threshold * 2: color = [1, 0, 0]
        elif distance > threshold * 1.5: color = [1, 0.65, 0]
        else: color = [1, 1, 0]
        defect_points.append((point, color))
    distances.append(distance.numpy()[0])

distances = np.array(distances)
defect = np.any(distances > threshold)

defect_cloud = o3d.geometry.PointCloud()
for point in defect_points:
    defect_cloud.points.append(point[0])
    defect_cloud.colors.append(point[1])
o3d.visualization.draw_geometries([defect_cloud, source_temp])

print(np.max(distances))
print('Defect' if defect else 'No Defect')

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
0.024427515
Defect
